## CS431/631 Big Data Infrastructure
### Winter 2023 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Ananya Trivedi
* **ID:** 20874394

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



Run the next block to download the text file (`Shakespeare.txt`) and the Python tokenizer module (`simple_tokenize.py`).

In [3]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of
*distinct* token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$. Note that we're considering things one line at a time, so the number of unique pairs is NOT (n)(n-1) where n is the number of unique tokens! If a line is "this is fine" then there are 6 distinct pairs on that line.

As a sanity check, when rounded you should be getting ~26K distinct tokens and ~2M distinct pairs. (Please remember I ROUNDED so e..g if you say "I got 1.9M is that OK" then I will worry about you)

In [4]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
from itertools import combinations

tokens = set()
pairs = set()

with open('Shakespeare.txt') as f:
    for line in f:
        t = simple_tokenize(line)

        # add each token to the set of distinct tokens
        tokens.update(t)

        # all possible token pairs for the line
        for pair in combinations(t, 2):
            # ignore (x, x)
            if pair[0] != pair[1]:
                # add both (x, y) and (y, x) to the set
                pairs.add(pair)
                pairs.add((pair[1], pair[0]))
print(f"Number of distinct tokens: {len(tokens)}")
print(f"Number of distinct token pairs: {len(pairs)}")

# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

Number of distinct tokens: 25975
Number of distinct token pairs: 1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.

Please appropriately handle cases where the token being queried isn't in the file. Print an appropriate error message and continue reading input.
(We won't mark the text of your error messages, but it's bad practice to crash if someone has a typo)

Sanity checks:

```
Input 1 or 2 space-separated tokens (return to quit): the end
n(the,end) = 157
PMI(the,end) = 0.3505058356267105
```

If you got n=147 then you didn't use `simple_tokenize()`, you used `split()` and were thrown off by puntuation and capitlization.

(If you haven't been told this before, floating point numbers are inexact. You do not need to get exactly those digits, but should match at least 4 of them! Make sure you used log base 10)

If your PMI is 2.9ish then you probably are counting how many times each token occurrs rather than how many LINES contain that token.

Because of the phrase "at rope's end", you should also expect that "rope's" doesn't really show up outside this phrase, so it will have a high PMI with "end".

```
n(end,rope's) = 5, PMI(end,rope's) = 2.5ish
```


In [11]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log
from itertools import combinations
from collections import defaultdict

# Data structures
nx = defaultdict(int)  # To store n(x)
nxy = defaultdict(int)   # To store n(x, y)
line_count = 0           # Total number of lines

# Read and process the Shakespeare.txt file
with open('Shakespeare.txt') as f:
    for line in f:
        line_count += 1
        tokens = set(simple_tokenize(line))  # Tokenize and remove duplicates in the line

        # Update the count for each token
        for token in tokens:
            nx[token] += 1

        # Update the count for each token pair
        for pair in combinations(tokens, 2):
            nxy[pair] += 1
            nxy[(pair[1], pair[0])] += 1  # Add both (x, y) and (y, x)

# PMI calculation function
def pmi_calculation(x, y, nx, nxy, line_count):
    if x not in nx or y not in nx:
        return None

    NX = nx[x]
    NY = nx[y]
    NXY = nxy.get((x, y), 0)

    if NXY == 0:
        return None

    PX = NX / line_count
    PY = NY / line_count
    PXY = NXY / line_count

    pmi = log(PXY / (PX * PY), 10)
    return pmi

# One-token query function
def one_token(x, threshold, nx, nxy, line_count):
    if x not in nx:
        return f"Token '{x}' not found in file"

    NX = nx[x]
    pmi_vals = []
    for y in nx:
        if y != x:  # Ignore reflexive pairs (x, x)
            pmi = pmi_calculation(x, y, nx, nxy, line_count)
            if pmi is not None and nxy[(x, y)] >= threshold:
                pmi_vals.append((y, pmi, nxy[(x, y)]))

    pmi_vals.sort(key=lambda pair: pair[1], reverse=True)
    top5pmi = pmi_vals[:5]

    return NX, top5pmi

# Two-token query function
def two_token(x, y, nx, nxy, line_count):
    if x not in nx or y not in nx:
        return None, None

    NXY = nxy.get((x, y), 0)
    if NXY == 0:
        return NXY, None

    pmi = pmi_calculation(x, y, nx, nxy, line_count)
    return NXY, pmi


while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)

    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue

        # ONE TOKEN QUERY:
        NX, top5pmi = one_token(q_tokens[0], threshold, nx, nxy, line_count)

        # Output result in the format of the print statements
        print("  n({}) = {}".format(q_tokens[0], NX))
        print("  high PMI tokens with respect to {} (threshold: {}):".format(q_tokens[0], threshold))

        for i, (token, pmi, NXY) in enumerate(top5pmi):
            print("    n({}, {}) = {}, PMI({}, {}) = {:.5f}".format(q_tokens[0], token, NXY, q_tokens[0], token, pmi))

    elif len(q_tokens) == 2:
        # TWO TOKEN QUERY:
        NXY, pmi = two_token(q_tokens[0], q_tokens[1], nx, nxy, line_count)
        if NXY:
            print("  n({}, {}) = {}".format(q_tokens[0], q_tokens[1], NXY))  # Both tokens
            print("  PMI({}, {}) = {:.5f}".format(q_tokens[0], q_tokens[1], pmi))  # PMI value
        else:
            print(f"Tokens '{q_tokens[0]}' and '{q_tokens[1]}' do not co-occur.")

    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): the
Input a positive integer frequency threshold: 4
  n(the) = 24300
  high PMI tokens with respect to the (threshold: 4):
    n(the, proudest) = 14, PMI(the, proudest) = 0.70238
    n(the, calendar) = 7, PMI(the, calendar) = 0.70238
    n(the, bee) = 10, PMI(the, bee) = 0.70238
    n(the, ninth) = 7, PMI(the, ninth) = 0.70238
    n(the, thracian) = 5, PMI(the, thracian) = 0.70238
Input 1 or 2 space-separated tokens (return to quit): the end
  n(the, end) = 157
  PMI(the, end) = 0.35051
Input 1 or 2 space-separated tokens (return to quit): 


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

Consider two seperate parts: The initial processing of the file, and then the time required to run the queries themselves.
(If you did not do any initial processing and reloaded everything for each query, then please go fix that, that's not okay.)


In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

(Tip since we probably haven't got that far in lectures yet: Vocabulary size for a document actually grows unbounded, proportional to the log of the document size!)

#### Answer to Question 3:

Let us break down the code into components and analyze how the execution would change if we try to run the PMI analysis on larger files.

Initial Processing of the File:
For each line, we are extracting the tokens; its execution depends on the number of tokens in the line. For each token:
*   we update the `nx` dictionary (which takes `O(1)` time per token T, which takes `O(T)` time)
*   and also the `nyx` dictionary (with a total of `T*(T-1)/2` pairs formed from T tokens, which is ~`O(T^2)` pairs).


Now let us consider the overall complexity. Let's say we have a total of `N` lines in the file. The total complexity will then be `O(NT^2)` where T is the average number of tokens per line (vocabulary size or the number of distinct tokens)

This means that for each line, the time to update token counts (`nx`) is `O(T)`, and the time to update pair counts (`nxy`) is `O(T^2)`. Summing this over all N lines, the overall time complexity for the initial processing step is `O(NT^2)`.

T does not increase linearly with the document size. As mentioned in the question, vocabulary size grows sublinearly, roughly proportional to the logarithm of the document size. So, as the document size increases, T will grow, but at a slower rate than the number of lines N.

---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.